# 08 - Vertex AI Custom Model - R - From Notebook to Script

# <IN ACTIVE DEVELOPMENT - NOT COMPLETE>
    
Dev Notes:
- Python Kernel:
    - use notebook instance with R kernel
    - or, install R here with needed libraries
    
Workflow:
- Construct R script for creating model with annotations
- Test R script at command line
   



---
## Setup

inputs:

In [2]:
SCRIPT_NAME = 'train.R'

In [3]:
SCRIPT_PATH = './code/' + SCRIPT_NAME

packages:

In [4]:
import os
from IPython.display import Markdown as md

environment:

In [5]:
if os.path.exists('code'):
    print('The code directory alredy exists')
else:
    print('Creating the code directory')
    os.makedirs('code')

Creating the code directory


---
## Build the ML Training Code
Use the cell magic `%%writefile` to create a single ML training code file named `train.R`.

### Import Libraries

In [6]:
%%writefile {SCRIPT_PATH}

# library import
library(bigrquery)
library(dplyr)

Writing ./code/train.R


### Parse Command Line Inputs

In [7]:
%%writefile -a {SCRIPT_PATH}

# inputs
args <- commandArgs(trailingOnly = TRUE)
project_id <- args[1]
region <- args[2]
experiment <- args[3]
series <- args[4]
bq_project <- args[5]
bq_dataset <- args[6]
bq_table <- args[7]
var_target <- args[8]
var_omit <- args[9]

Appending to ./code/train.R


### Retrieve Data From BigQuery

In [8]:
%%writefile -a {SCRIPT_PATH}

# data source
get_data <- function(s){
    query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "%s"', var_omit, bq_project, bq_dataset, bq_table, s)
    table <- bq_project_query(bq_project, query)
    ds <- bq_table_download(table)
    return(ds)
}
train <- get_data("TRAIN")
test <- get_data("TEST")

Appending to ./code/train.R


### Fit Model

In [9]:
%%writefile -a {SCRIPT_PATH}

# logistic regression model
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

Appending to ./code/train.R


### Predictions for Test Split

In [10]:
%%writefile -a {SCRIPT_PATH}

# predictions for evaluation
preds <- predict(model, test, type = "response")

Appending to ./code/train.R


### Evaluate The Model

In [11]:
%%writefile -a {SCRIPT_PATH}

# evaluate
actual <- test[, var_target]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
results <- cbind(actual, pred)
cm <- table(results)

Appending to ./code/train.R


### Save The Model

In [12]:
%%writefile -a {SCRIPT_PATH}

# save model to file
saveRDS(model, "model.rds")

Appending to ./code/train.R


---
## Review The Training Code
Read in the completed script and print it out for review below:

In [13]:
with open(SCRIPT_PATH, 'r') as file:
    data = file.read()
md(f"```r\n\n{data}\n```")

```r


# library import
library(bigrquery)
library(dplyr)

# inputs
args <- commandArgs(trailingOnly = TRUE)
project_id <- args[1]
region <- args[2]
experiment <- args[3]
series <- args[4]
bq_project <- args[5]
bq_dataset <- args[6]
bq_table <- args[7]
var_target <- args[8]
var_omit <- args[9]

# data source
get_data <- function(s){
    query = sprintf('SELECT * EXCEPT(%s, splits) FROM `%s.%s.%s` WHERE splits = "%s"', var_omit, bq_project, bq_dataset, bq_table, s)
    table <- bq_project_query(bq_project, query)
    ds <- bq_table_download(table)
    return(ds)
}
train <- get_data("TRAIN")
test <- get_data("TEST")

# logistic regression model
model <- glm(
    Class ~ .,
    data = train,
    family = binomial)

# predictions for evaluation
preds <- predict(model, test, type = "response")

# evaluate
actual <- test[, var_target]
names(actual) <- 'actual'
pred <- tibble(round(preds))
names(pred) <- 'pred'
results <- cbind(actual, pred)
cm <- table(results)

# save model to file
saveRDS(model, "model.rds")

```

## Test Script

Requires an R enviornment locally with correct libraries installed.  This will run the script and result in a save model file (`model.rds`).

In [76]:
!cd ./code && Rscript {SCRIPT_NAME} statmike-mlops-349915 us-central1 08 08 statmike-mlops-349915 fraud fraud_prepped Class transaction_id


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
glm.fit: fitted probabilities numerically 0 or 1 occurred 
